In [247]:
from sklearn import datasets
import sys
import numpy as np

In [248]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [249]:
print type(X), type(y)
print X[:2]
print y[:0]

<type 'numpy.ndarray'> <type 'numpy.ndarray'>
[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]]
[]


In [250]:
class Split(object):
    def __init__(self, feature_id, threshold):
        self.feature_id = feature_id
        self.threshold = threshold
        
    def is_greater(self, example):
        return example[self.feature_id] > self.threshold
        
    def get_split(self, X, y):
        
        X_left = []
        y_left = []
        X_right = []
        y_right = []
        
        for i in xrange(len(X)):
            if self.is_greater(X[i]):
                X_right.append(X[i])
                y_right.append(y[i])
            else:
                X_left.append(X[i])
                y_left.append(y[i])
        
        return X_left, y_left, X_right, y_right

class Node(object):
    def __init__(self, feature_id, threshold):
        self.feature_id = feature_id
        self.threshold = threshold
        self.isLeaf = False
        self.resultTarget = None
        self.left = None
        self.right = None
        
    def toString(self, i):
        res = ""
        if self.isLeaf:
            res = res + " " * i + "Leaf. Result: " + str(self.resultTarget) + "\n"
        else:
            res = res + " " * i + "Feature: " + str(self.feature_id) + "\n" + " " * i + \
                    "Threshold: " + str(self.threshold) + "\n"
            res = res + self.left.toString(i + 2)
            res = res + self.right.toString(i + 2)
        return res
            
    def predict(self, x):
        if self.isLeaf:
            return self.resultTarget
        if x[self.feature_id] < self.threshold:
            return self.left.predict(x)
        else:
            return self.right.predict(x)
        
class DecisionTreeClassifier(object):
    
    def __init__(self):
        self.root = None
    
    def toString(self):
        return self.root.toString(1)
    
    def fit(self, X, y):
        self.root = self.make_node(X, y)
        
    def make_node(self, X, y):
        if len(set(y)) == 1:
            node = Node(None, None)
            node.isLeaf = True
            node.resultTarget = y[0]
            return node
        
        best_g = sys.maxint
        best_feature = -1
        best_threshold = -1
        X = np.array(X)
        for i in xrange(len(X[0])):
            tresholds = list(set(X[:, i]))
            tresholds.sort()
            for t_m in [(tresholds[j] + tresholds[j + 1])/2 for j in range(len(tresholds) - 1)]:
                split = Split(i, t_m)
                X_l, y_l, X_r, y_r = split.get_split(X, y)
                g = self.G(X_l, y_l, X_r, y_r)
                if g < best_g:
                    best_g = g
                    best_feature = i
                    best_threshold = t_m
                    
        node = Node(best_feature, best_threshold)
        
        split = Split(best_feature, best_threshold)
        X_l, y_l, X_r, y_r = split.get_split(X, y)
        node.left = self.make_node(X_l, y_l)
        node.right = self.make_node(X_r, y_r)
        node.isLeaf = False
        return node
    
                
    def G(self, X_l, y_l, X_r, y_r):
        self.N = len(X_l) + len(X_r)
        return float(len(X_l)) / self.N * self.H(y_l) + float(len(X_r)) / self.N * self.H(y_r)
        
    def H(self, y):
        res = 0
        for k in set(y):
            res = res + self.p_k(y, k) * (1 - self.p_k(y, k))
        return res
    
    def p_k(self, y, k):
        res = 0
        for i in y:
            if i == k:
                res = res + 1
        return float(res) / self.N
        
    def predict(self, x):
        return self.root.predict(x)
        

In [251]:
clf = DecisionTreeClassifier()
clf.fit(X, y)

In [252]:
print clf.toString()

 Feature: 2
 Threshold: 4.35
   Feature: 2
   Threshold: 2.45
     Leaf. Result: 0
     Leaf. Result: 1
   Feature: 3
   Threshold: 1.65
     Feature: 2
     Threshold: 4.95
       Leaf. Result: 1
       Feature: 0
       Threshold: 6.05
         Feature: 1
         Threshold: 2.45
           Leaf. Result: 2
           Leaf. Result: 1
         Leaf. Result: 2
     Feature: 3
     Threshold: 1.75
       Feature: 0
       Threshold: 5.8
         Leaf. Result: 2
         Leaf. Result: 1
       Feature: 2
       Threshold: 4.85
         Feature: 0
         Threshold: 5.95
           Leaf. Result: 1
           Leaf. Result: 2
         Leaf. Result: 2



In [253]:
print X[0]
print "True: ", y[0]
print "Predict: ", clf.predict(X[0])

[ 5.1  3.5  1.4  0.2]
True:  0
Predict:  0


In [254]:
print X[-1]
print "True: ", y[-1]
print "Predict: ", clf.predict(X[-1])

[ 5.9  3.   5.1  1.8]
True:  2
Predict:  2


In [255]:
print X[50]
print "True: ", y[50]
print "Predict: ", clf.predict(X[50])

[ 7.   3.2  4.7  1.4]
True:  1
Predict:  1


In [256]:
X_test = []
y_test = []
X_fit = []
y_fit = []

for i in range(0, len(X), 2):
    X_test.append(X[i])
    y_test.append(y[i])

for i in range(1, len(X), 2):
    X_fit.append(X[i])
    y_fit.append(y[i])

In [257]:
clf = DecisionTreeClassifier()
clf.fit(X_fit, y_fit)

In [258]:
y_predict = [clf.predict(X_test[i]) for i in range(len(X_test))]

In [259]:
quality = sum(np.array(y_predict) == np.array(y_test)) / float(len(y_test))

print "Quality: ", quality

Quality:  0.933333333333
